In [2]:
!pwd

/cygdrive/c/Users/pat/Documents/Online_Training/MachineLearningTraining/AWS Certified Machine Learning class/tutorials/Hands-on_AI_SageMaker_Course/AWS_Sagemaker_Course-main


In [1]:
#%load_ext nb_black

<IPython.core.display.Javascript object>

In [3]:
import os
import json
import random
import pandas as pd
import shutil

In [6]:
base_path="flower_photos_90/img_clf_lst/"
subdirs = ["train_imgs", "valid_imgs", "train_annots", "valid_annots"]
for subdir in subdirs:
    path = base_path + "/" + subdir
    if not os.path.exists(path):
        print(f"creating directory {path}")
        os.makedirs(path)
    else:
       print(f"{path} already exist")

flower_photos_90/img_clf_lst//train_imgs already exist
creating directory flower_photos_90/img_clf_lst//valid_imgs
creating directory flower_photos_90/img_clf_lst//train_annots
creating directory flower_photos_90/img_clf_lst//valid_annots


In [7]:
gt_df = pd.read_csv(
    "flower_photos_90/img_clf_lst/all_images_gt/clf_labels.csv", header=None
)
gt_df.columns = ["image", "class"]
gt_df.head()

,image,class
0,100080576_f52e8ee070_n.jpg,[daisy]
1,10140303196_b88d3d6cec.jpg,[daisy]
2,10172379554_b296050f82_n.jpg,[daisy]
3,10172567486_2748826a8b.jpg,[daisy]
4,10172636503_21bededa75_n.jpg,[daisy]


In [8]:
train_valid_split = 0.7
nimages = gt_df["image"].nunique()
ntrain = int(train_valid_split * nimages)
nvalid = nimages - ntrain
print(nimages, ntrain, nvalid)

90 62 28


In [9]:
gt_df["class"] = gt_df["class"].apply(lambda x: x[1:-1])
gt_df.head()

,image,class
0,100080576_f52e8ee070_n.jpg,daisy
1,10140303196_b88d3d6cec.jpg,daisy
2,10172379554_b296050f82_n.jpg,daisy
3,10172567486_2748826a8b.jpg,daisy
4,10172636503_21bededa75_n.jpg,daisy


In [10]:
labels = ["daisy", "dandelion", "roses"]
labels_map = {k: v for v, k in enumerate(labels)}
with open("flower_photos_90/img_clf_lst/labels_map.json", "w") as fp:
    json.dump(labels_map, fp)
labels_map

{'daisy': 0, 'dandelion': 1, 'roses': 2}

In [11]:
gt_df["class_id"] = gt_df["class"].map(labels_map)
gt_df.head()

,image,class,class_id
0,100080576_f52e8ee070_n.jpg,daisy,0
1,10140303196_b88d3d6cec.jpg,daisy,0
2,10172379554_b296050f82_n.jpg,daisy,0
3,10172567486_2748826a8b.jpg,daisy,0
4,10172636503_21bededa75_n.jpg,daisy,0


In [12]:
gt_df = gt_df.sample(frac=1).reset_index(drop=True)
gt_df.head()

,image,class,class_id
0,12406229175_82e2ac649c_n.jpg,roses,2
1,10770585085_4742b9dac3_n.jpg,daisy,0
2,10466290366_cc72e33532.jpg,daisy,0
3,10828951106_c3cd47983f.jpg,dandelion,1
4,11124324295_503f3a0804.jpg,daisy,0


In [13]:
gt_df["index"] = gt_df.index + 1
gt_df.head()

,image,class,class_id,index
0,12406229175_82e2ac649c_n.jpg,roses,2,1
1,10770585085_4742b9dac3_n.jpg,daisy,0,2
2,10466290366_cc72e33532.jpg,daisy,0,3
3,10828951106_c3cd47983f.jpg,dandelion,1,4
4,11124324295_503f3a0804.jpg,daisy,0,5


In [14]:
sel_cols = ["index", "class_id", "image"]
gt_df[sel_cols].head(ntrain).to_csv(
    "flower_photos_90/img_clf_lst/train_annots/train.lst",
    sep="\t",
    index=False,
    header=False,
)
!head -n 5 flower_photos_90/img_clf_lst/train_annots/train.lst

1	2	12406229175_82e2ac649c_n.jpg
2	0	10770585085_4742b9dac3_n.jpg
3	0	10466290366_cc72e33532.jpg
4	1	10828951106_c3cd47983f.jpg
5	0	11124324295_503f3a0804.jpg


In [15]:
gt_df[sel_cols].tail(nvalid).to_csv(
    "flower_photos_90/img_clf_lst/valid_annots/valid.lst",
    sep="\t",
    index=False,
    header=False,
)
!head -n 5 flower_photos_90/img_clf_lst/valid_annots/valid.lst

63	0	10993818044_4c19b86c82.jpg
64	2	12407768513_3440238148_n.jpg
65	1	11296320473_1d9261ddcb.jpg
66	1	11545123_50a340b473_m.jpg
67	1	1080179756_5f05350a59.jpg


In [16]:
train_df = pd.read_csv(
    "flower_photos_90/img_clf_lst/train_annots/train.lst", sep="\t", header=None
)
images = list(train_df[2].values)
for image in images:
    shutil.copy(
        "flower_photos_90/img_clf_lst/all_images_gt/" + image,
        "flower_photos_90/img_clf_lst/train_imgs/",
    )

In [17]:
valid_df = pd.read_csv(
    "flower_photos_90/img_clf_lst/valid_annots/valid.lst", sep="\t", header=None
)
images = list(valid_df[2].values)
for image in images:
    shutil.copy(
        "flower_photos_90/img_clf_lst/all_images_gt/" + image,
        "flower_photos_90/img_clf_lst/valid_imgs/",
    )

<IPython.core.display.Javascript object>